In [26]:
import pandas as pd
import numpy as np

df = pd.read_excel('G:\\Drives compartilhados\\Finance 3.0\\1. Tax\\Fiscal Indiretos\\FILIAL L4B 0001 SP - ATIVA\\DMS\\Importação em lote planilha\\Arquivos\\EXPORT.xlsx', 
                   dtype=str)
cod_sp = pd.read_excel('G:\\Drives compartilhados\\Finance 3.0\\1. Tax\\Fiscal Indiretos\\FILIAL L4B 0001 SP - ATIVA\DMS\\Importação em lote planilha\\Arquivos\\CODSP.xlsx',
                       dtype=str)
c_Prestadores = pd.read_excel('G:\\Drives compartilhados\\Finance 3.0\\1. Tax\\Fiscal Indiretos\\FILIAL L4B 0001 SP - ATIVA\DMS\\Importação em lote planilha\\Arquivos\\c_Prestadores.xlsx',
                       dtype=str)

df = df.rename(columns={'NºID fiscal': 'CNPJ', 'Número de nota fiscal eletrônica': 'Nota', 'Valor ISS':'ISS', 'LC 116 código de tipo de serviço':'LC 116'})

dtint = input('Data inicio: ')
dtfim = input('Data fim: ')
inMunicipal = input('Informar IM: ')
filial = input('Informe código da filial: ')

#Alterar tipo de dado da coluna.
df = df.astype({
    'CNPJ': str,
    'Nota': str,
    'ISS': float,
    'Valor': float
})

#Seleção Filial e filtros.
df = df.loc[df['Centro'] == filial]
df = df.loc[df['Cidade'] != 'SAO PAULO']
df = df[df['CFOP'].isin(['1933/AA', '2933/AA'])]
df['% ISS'] = df['% ISS'].str.rstrip('%').astype(float)

#Criação das colunas personalizadas, parâmetros conforme manual.
df.insert(3, 'T_Registro', '4')
df.insert(4, 'T_Documento', '02')
df.insert(5, 'Serie', '    E')
df.insert(6, 'SNFTS', 'N')
df.insert(7, 'Tributação', 'T')
df.insert(11, 'vDeducao', '000000000000000')
df.insert(4, 'Ind_CNPJ', '2')
df.insert(8, 'Regime_T', '0')
df.insert(11, 'Tipo_End', 'END')
df.insert(9, 'Complemento', ' ')
df.insert(12, 'CCM', '00000000')
df.insert(13, 'T_NFTS', '1')
df.insert(14, 'C_INSS', '0')
df.insert(15, 'N_OBRA', '0')
df.insert(15, 'C_RESERVADO', ' ')
df.insert(16, 'Data_PGTO', ' ')
df.insert(16, 'Descrição do Serviço', 'Contratação de Serviços')

#Criação das colunas conforme regra do valor.
def definir_iss_retido(row):
    """
        Define o valor do 'ISS Retido' para uma linha. Args:
        row (pandas.Series): Uma linha do DataFrame. Returns:
        str: O valor do 'ISS Retido' para a linha.
    """
    if row['ISS'] > 0:
        return '1'
    elif row['ISS'] == 0:
        return '2'
    # Retorne outros valores de acordo com suas necessidades
    else:
        return np.NAN  # Valor padrão para linhas sem 'ISS' > 0 ou 'ISS' == 0

df['ISS Retido'] = df.apply(definir_iss_retido, axis=1)  # Aplicar a função a cada linha
df['Valor'] = df['Valor'].apply(lambda x: format(round(x, 2), '.2f'))
df['% ISS'] = (df['% ISS'] * 100).round().astype(int)

#União das planilha base trazendo os subitens COD.
df = pd.merge(df, cod_sp, on='LC 116', how='inner')
df = pd.merge(df, c_Prestadores, on='CNPJ', how='left')

df.drop_duplicates(['Documento MIRO'], inplace=True)

# Converter a coluna 'Data documento' para datetime
df['Data documento'] = pd.to_datetime(df['Data documento'], errors='coerce')

# Agora você pode formatar ou extrair a parte da data
df['Data documento'] = df['Data documento'].dt.strftime('%Y%m%d')

df['Data documento'] = df['Data documento'].str.replace('-', '')
df['LC 116'] = df['LC 116'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('-', '')
df['Valor'] = df['Valor'].astype(str).str.replace('.', '')

#Parametrização das colunas utilizando 'PAD'
df['Nota'] = df['Nota'].str.pad(width=12, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Valor'] = df['Valor'].str.pad(width=15, side='left', fillchar='0')
df['CÓDIGO'] = df['CÓDIGO'].str.pad(width=5, side='left', fillchar='0')
df['% ISS'] = df['% ISS'].apply(lambda x: str(x).zfill(4))
df['Razão Social'] = df['Razão Social'].str.pad(width=75, side='left', fillchar=' ')
df['Logradouro'] = df['Logradouro'].str.pad(width=50, side='right', fillchar=' ')
df['Número'] = df['Número'].str.pad(width=10, side='right', fillchar=' ')
df['Complemento'] = df['Complemento'].str.pad(width=30, side='right', fillchar=' ')
df['Bairro'] = df['Bairro'].str.pad(width=30, side='right', fillchar=' ')
df['Cidade_y'] = df['Cidade_y'].str.pad(width=50, side='right', fillchar=' ')
df['E-mail'] = df['E-mail'].astype(str).apply(lambda x: x.ljust(75))
df['C_INSS'] = df['C_INSS'].str.pad(width=12, side='right', fillchar='0')
df['N_OBRA'] = df['N_OBRA'].str.pad(width=12, side='right', fillchar='0')
df['C_RESERVADO'] = df['C_RESERVADO'].str.pad(width=200, side='right', fillchar=' ')
df['Descrição do Serviço'] = df['Descrição do Serviço'].str.pad(width=200, side='right', fillchar=' ')
df['Data_PGTO'] = df['Data_PGTO'].str.pad(width=8, side='right', fillchar=' ')
df['UF'] = df['UF'].str.pad(width=2, side='right', fillchar=' ')
df['CEP'] = df['CEP'].str.pad(width=8, side='right', fillchar='0')

cabecalho = [f'1002{inMunicipal}{dtint}{dtfim}']
Header = pd.DataFrame(columns=cabecalho)

registros = pd.DataFrame()
registros = pd.concat([df['T_Registro'], df['T_Documento'], df['Serie'], df['Nota'], df['Data documento'], df['SNFTS'],
                       df['Tributação'], df['Valor'], df['vDeducao'], df['CÓDIGO'], df['LC 116'], df['% ISS'],
                       df['ISS Retido'], df['Ind_CNPJ'], df['CNPJ'], df['CCM'], df['Razão Social'], df['Tipo_End'], 
                       df['Logradouro'],df['Número'], df['Complemento'], df['Bairro'], df['Cidade_y'], df['UF'], df['CEP'], 
                       df['E-mail'],df['T_NFTS'], df['Regime_T'], df['Data_PGTO'], df['C_INSS'], df['N_OBRA'], df['C_RESERVADO'],
                       df['Descrição do Serviço']],axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

df = df.astype({'Valor': int, })
num_linhas = len(registros)
num_linhas_formatted = f'{num_linhas:0>6}'
Total = df['Valor'].sum()
Total = str(Total)
TotalZ = Total.zfill(15)

radape = ['90' + num_linhas_formatted + TotalZ + '000000000000000']
footer = pd.DataFrame(columns=radape)

#Escrever linha por linha no arquivo de texto
with open(f'C:\\Users\\logger\\Downloads\\NFTS\\{filial}.txt', 'w', encoding='utf-8') as file:
    #Iterar sobre todos os DataFrames (df1, df2, df3) dinamicamente
    for df_name in ['Header', 'registros', 'footer']:
        df = globals()[df_name]  #Obtém o DataFrame pelo nome da variável
        df.to_csv(file, index=False, sep='\t')  #Escreve o DataFrame no arquivo


<>:6: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:6: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
C:\Users\logger\AppData\Local\Temp\ipykernel_10916\2905090920.py:6: SyntaxWarning: invalid escape sequence '\D'
  cod_sp = pd.read_excel('G:\\Drives compartilhados\\Finance 3.0\\1. Tax\\Fiscal Indiretos\\FILIAL L4B 0001 SP - ATIVA\DMS\\Importação em lote planilha\\Arquivos\\CODSP.xlsx',
C:\Users\logger\AppData\Local\Temp\ipykernel_10916\2905090920.py:8: SyntaxWarning: invalid escape sequence '\D'
  c_Prestadores = pd.read_excel('G:\\Drives compartilhados\\Finance 3.0\\1. Tax\\Fiscal Indiretos\\FILIAL L4B 0001 SP - ATIVA\DMS\\Importação em lote planilha\\Arquivos\\c_Prestadores.xlsx',
C:\Users\logger\AppData\Local\Temp\ipykernel_10916\2905090920.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [27]:
df = pd.read_excel('G:\\Drives compartilhados\\Finance 3.0\\1. Tax\\Fiscal Indiretos\\FILIAL L4B 0001 SP - ATIVA\\DMS\\Importação em lote planilha\\SAP Novo v1.1.xlsx', 
                   dtype=str, sheet_name='São José')

competencia = input('Informe a competência (MM/AAAA): ')
codFilial = input('Informe o Código da Filial: ') #Conforme livro do SAP

  #Alterar tipo de dado da coluna.
df = df.astype({
      'CNPJ': str,
      'Nota': str,
      'ISS': float,
      'Valor': float,
      'Logradouro': str
  })

if codFilial == '0087':
      filialCnpj = '24217653008846'

  #Seleção Filial e filtros.
df = df.loc[df['Centro'] == codFilial]
df = df.loc[df['Cidade'] != 'SAO JOSE']
df = df[df['CFOP'].isin(['1933/AA', '2933/AA'])]
df['%ISS'] = df['%ISS'].str.rstrip('%').astype(float)

  #Criação das colunas personalizadas, parâmetros conforme manual.
df.insert(1, 'T_Registro10', '10')
df.insert(2, 'T_Registro20', '20')
df.insert(2, 'T_Registro30', '30')
df.insert(3, 'T_Serviço', '2')
df.insert(4, 'T_Documento', '05')
df.insert(5, 'Competencia', competencia)
df.insert(6, 'T_PessoaP', 'J')
df.insert(7, 'T_PessoaT', 'J')
df.insert(8, 'Situaçao', 'E')
df.insert(9, 'CNPJT', filialCnpj)
df.insert(10, 'Simples', 'N')
df.insert(11, 'Sep', ';')
df.insert(12, 'C_RESERVADO', '')
df.insert(13, 'RISSOBRA', '0')
df.insert(13, 'Deducao', 0.00)
df.insert(14, 'TPrestador', 'N')
df.insert(15, 'T_Servico', '2')
df.insert(16, 'Complemento', ' ')
df.insert(17, 'Fone', ' ')
df.insert(16, 'Observacoes', ' ')

df['ISS Retido'] = df.apply(lambda row: '01' if row['ISS'] > 0 else '14', axis=1)

df['Data documento'] = df['Data documento'].str.replace('-', '/')
df['Data documento'] = pd.to_datetime(df['Data documento'])
df['Data documento'] = df['Data documento'].dt.strftime('%d/%m/%Y')
df['CEP'] = df['CEP'].str.replace('.', '')
df['CEP'] = df['CEP'].str.replace('-', '')
df['LC 116'] = df['LC 116'].str.replace('.', '')
df['%ISS'] = df['%ISS'].replace(0.00, 3.00)

  #Criação das colunas conforme regra do valor.
df['Valor'] = df['Valor'].apply(lambda x: format(round(x, 2), '.2f'))
df['Deducao'] = df['Deducao'].apply(lambda x: format(round(x, 2), '.2f'))
df['ISS'] = df['ISS'].apply(lambda x: format(round(x, 2), '.2f'))
df['%ISS'] = df['%ISS'].apply(lambda x: format(round(x, 2), '.2f'))

  #Parametrização das colunas utilizando 'PAD'
df['Nota'] = df['Nota'].str.pad(width=15, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Código municipios'] = df['Código municipios'].str.pad(width=7, side='left', fillchar='0')
df['RISSOBRA'] = df['RISSOBRA'].str.pad(width=10, side='left', fillchar='0')
df['LC 116'] = df['LC 116'].str.pad(width=7, side='left', fillchar='0')
df['%ISS'] = df['%ISS'].apply(lambda x: str(x).zfill(6))
df['ISS'] = df['ISS'].apply(lambda x: str(x).zfill(18))
df['Deducao'] = df['Deducao'].apply(lambda x: str(x).zfill(18))
df['Valor'] = df['Valor'].apply(lambda x: str(x).zfill(18))
df['Razão Social'] = df['Razão Social'].str.pad(width=40, side='left', fillchar=' ')
df['Logradouro'] = df['Logradouro'].str.pad(width=40, side='right', fillchar=' ')
df['Número'] = df['Número'].str.pad(width=6, side='right', fillchar=' ')
df['Bairro'] = df['Bairro'].str.pad(width=20, side='right', fillchar=' ')
df['Cidade'] = df['Cidade'].str.pad(width=30, side='left', fillchar=' ')
df['UF'] = df['UF'].str.pad(width=2, side='right', fillchar=' ')
df['CEP'] = df['CEP'].str.pad(width=8, side='right', fillchar='0')
df['Complemento'] = df['Complemento'].str.pad(width=20, side='right', fillchar=' ')
df['Observacoes'] = df['Observacoes'].str.pad(width=100, side='right', fillchar=' ')
df['Fone'] = df['Fone'].str.pad(width=12, side='right', fillchar=' ')

RG10 = pd.DataFrame()
RG10 = pd.concat([df['T_Registro10'], df['Sep'], df['T_Serviço'],df['Sep'], df['T_Documento'], df['Sep'], df['Nota'], df['Sep'],
                      df['Competencia'], df['Sep'], df['T_PessoaP'], df['Sep'], df['CNPJ'], df['Sep'], df['T_PessoaT'], df['Sep'],
                      df['CNPJT'], df['Sep'], df['Data documento'], df['Sep'], df['Valor'], df['Sep'], df['Situaçao'], df['Sep'],
                      df['Observacoes'], df['Sep'], df['Simples'], df['Sep']], axis=1,
                    ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

RG20 = pd.DataFrame()
RG20 = pd.concat([df['T_Registro20'], df['Sep'], df['T_Servico'], df['Sep'], df['T_Documento'], df['Sep'], df['Nota'], df['Sep'], df['Competencia'], df['Sep'],
                    df['T_PessoaP'], df['Sep'], df['CNPJ'], df['Sep'], df['T_PessoaT'], df['Sep'], df['CNPJT'], df['Sep'],
                    df['LC 116'], df['Sep'], df['%ISS'], df['Sep'], df['Valor'], df['Sep'], df['Deducao'], df['Sep'],
                    df['ISS'], df['Sep'], df['Código municipios'], df['Sep'], df['ISS Retido'], df['Sep'], df['TPrestador'], df['Sep'], df['RISSOBRA'], df['Sep']],
                  axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

RG30 = pd.DataFrame()
RG30 = pd.concat([df['T_Registro30'], df['Sep'], df['T_PessoaP'], df['Sep'], df['CNPJ'], df['Sep'],
                    df['Razão Social'], df['Sep'], df['Logradouro'], df['Sep'], df['Número'], df['Sep'],
                    df['Complemento'], df['Sep'], df['Bairro'], df['Sep'], df['Cidade'], df['Sep'],
                    df['UF'], df['Sep'], df['CEP'], df['Sep'], df['Fone'], df['Sep'], df['Fone'],df['Sep']],axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

interleaved_dfs = []

for i in range(len(RG10)):
      interleaved_dfs.append(RG10.iloc[i])
      interleaved_dfs.append(RG20.iloc[i])
      interleaved_dfs.append(RG30.iloc[i])
      
result = pd.DataFrame(interleaved_dfs)

result.to_csv('C:\\Users\\logger\\Downloads\\NFTS\\NFTS_SJ.txt', index=False, header=False)

In [32]:
import pandas as pd

df = pd.read_excel('C:\\Users\\logger\\Downloads\\Importacao.xltx', dtype=str)

df = df.astype({
    'Valor R$': float,
    '%ISS': float
})

# Datas em formato americano.
dtint = input('Data inicio: ')
dtfim = input('Data fim: ')
inMunicial = input('Informar IM: ')
filial: str = input('Informe código da filial: ')

# Filtros
df = df.loc[df['Empresa'] == filial]
df = df[df['NFTS SP'].isnull()]

df.insert(3, 'T_Registro', '4')
df.insert(4, 'T_Documento', '01')
df.insert(5, 'Serie', 'INV')
df.insert(6, 'SNFTS', 'N')
df.insert(7, 'Tributação', 'T')
df.insert(11, 'vDeducao', '000000000000000')
df.insert(4, 'Ind_CNPJ', '3')
df.insert(8, 'Regime_T', '0')
df.insert(11, 'Tipo_End', ' ')
df.insert(9, 'Complemento', ' ')
df.insert(12, 'CCM', '00000000')
df.insert(13, 'T_NFTS', '1')
df.insert(14, 'C_INSS', '0')
df.insert(15, 'N_OBRA', '0')
df.insert(15, 'C_RESERVADO', ' ')
df.insert(16, 'Data_PGTO', ' ')
df.insert(17, 'ISS Retido', '1')
df.insert(18, 'CNPJ', '0')
df.insert(18, 'LOGRADOURO', 'EUA')
df.insert(18, 'NÚMERO', ' ')
df.insert(18, 'BAIRRO', ' ')
df.insert(18, 'CIDADE', ' ')
df.insert(18, 'UF', ' ')
df.insert(18, 'CEP', '0')
df.insert(18, 'EMAIL', ' ')
df.insert(18, 'Descrição do Serviço', 'Prestação de serviços do exterior.')

df['Valor R$'] = df['Valor R$'].apply(lambda x: format(round(x, 2), '.2f'))
df['%ISS'] = (df['%ISS'] * 100).round().astype(int)

df = df.astype({
    'Valor R$': str,
})

# df['%ISS'] = df['%ISS'].str.replace(',', '').str.replace('%', '')
df['Valor R$'] = df['Valor R$'].str.replace('.', '').str.replace(',', '').str.replace('R$', '')
df['Data Fechamento'] = df['Data Fechamento'].str.replace('/', '')
df['Cód. ISS'] = df['Cód. ISS'].str.replace('.', '')
df['Data Fechamento'] = pd.to_datetime(df['Data Fechamento'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y %m %d')
# df['Data Fechamento'] = pd.to_datetime(df['Data Fechamento'], format='%d%m%Y').dt.strftime('%Y %m %d')
df['Data Fechamento'] = df['Data Fechamento'].str.replace(' ', '')

# Parametrização das colunas utilizando 'PAD'
df['Invoice'] = df['Invoice'].str.pad(width=12, side='left', fillchar='0')
df['CNPJ'] = df['CNPJ'].str.pad(width=14, side='left', fillchar='0')
df['Valor R$'] = df['Valor R$'].str.pad(width=15, side='left', fillchar='0')
df['SUBITEM'] = df['SUBITEM'].str.pad(width=5, side='left', fillchar='0')
df['%ISS'] = df['%ISS'].apply(lambda x: str(x).zfill(4))
df['Fornecedor'] = df['Fornecedor'].astype(str).str.rjust(75)
df['LOGRADOURO'] = df['LOGRADOURO'].astype(str).str.rjust(50)
df['Serie'] = df['Serie'].astype(str).str.rjust(5)
df['NÚMERO'] = df['NÚMERO'].astype(str).str.rjust(12)
df['Complemento'] = df['Complemento'].astype(str).str.rjust(30)
df['BAIRRO'] = df['BAIRRO'].astype(str).str.rjust(30)
df['CIDADE'] = df['CIDADE'].astype(str).str.rjust(50)
df['EMAIL'] = df['EMAIL'].astype(str).str.rjust(75)
df['C_INSS'] = df['C_INSS'].str.pad(width=12, side='right', fillchar='0')
df['N_OBRA'] = df['N_OBRA'].str.pad(width=12, side='right', fillchar='0')
df['C_RESERVADO'] = df['C_RESERVADO'].astype(str).str.rjust(200)
df['Descrição do Serviço'] = df['Descrição do Serviço'].astype(str).str.rjust(200)
df['Data_PGTO'] = df['Data_PGTO'].str.pad(width=8, side='right', fillchar=' ')
df['UF'] = df['UF'].str.pad(width=2, side='right', fillchar=' ')
df['CEP'] = df['CEP'].str.pad(width=8, side='right', fillchar='0')

dfNFTS = pd.DataFrame()

# LTEC
dfNFTS[f'100259871091{dtint}{dtfim}'] = pd.concat(
    [df['T_Registro'], df['T_Documento'], df['Serie'], df['Invoice'], df['Data Fechamento'], df['SNFTS'],
     df['Tributação'], df['Valor R$'], df['vDeducao'], df['SUBITEM'], df['Cód. ISS'], df['%ISS'],
     df['ISS Retido'], df['Ind_CNPJ'], df['CNPJ'], df['CCM'], df['Fornecedor'], df['Tipo_End'], df['LOGRADOURO'],
     df['NÚMERO'], df['Complemento'], df['BAIRRO'], df['CIDADE'], df['UF'], df['CEP'], df['EMAIL'],
     df['T_NFTS'], df['Regime_T'], df['Data_PGTO'], df['C_INSS'], df['N_OBRA'], df['C_RESERVADO'],
     df['Descrição do Serviço']],
    axis=1, ignore_index=True).apply(lambda x: ''.join(str(val) for val in x), axis=1)

cabecalho = [f'1002{inMunicial}{dtint}{dtfim}']
Header = pd.DataFrame(columns=cabecalho)

df = df.astype({'Valor R$': int, })
num_linhas = len(dfNFTS)
num_linhas_formatted = f'{num_linhas:0>6}'
Total = df['Valor R$'].sum()
Total = str(Total)
TotalZ = Total.zfill(15)

radape = ['90' + num_linhas_formatted + TotalZ + '000000000000000']
footer = pd.DataFrame(columns=radape)

# Escrever linha por linha no arquivo de texto
with open(f'C:\\Users\\logger\\Downloads\\NFTS\\{filial}.txt', 'w', encoding='utf-8') as file:
    # Iterar sobre todos os DataFrames (df1, df2, df3) dinamicamente
    for df_name in ['Header', 'dfNFTS', 'footer']:
        df = globals()[df_name]  # Obtém o DataFrame pelo nome da variável
        df.to_csv(file, index=False, sep='\t')  # Escreve o DataFrame no arquivo
